In [1]:
#pip install NRCLex

In [6]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
# Import earning calls
import glob
list_articles = glob.glob("data/earning_call/*")
texts = []
earning_call_id = []
for s in list_articles:
    earning_call_id.append(s[-7:])
    with open(s) as f:
        texts.append(f.read())

In [8]:
# Get earning call id from file name
mypath= "data/earning_call"
from os import listdir
from os.path import isfile, join
earning_call_id = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [9]:
# Get dates
data_path = "data/date"
with open(data_path,"rb") as file :
    date = pickle.load(file)
print(len(date))
print(len(earning_call_id))

1200
1415


In [10]:
# Get tickers
data_path = "data/id_to_ticker"
with open(data_path,"rb") as file :
    id_mapping = pickle.load(file)

In [11]:
# Test : create dataframe with earning calls informations
df1 = pd.DataFrame(data={'id': earning_call_id})
df1['date']=earning_call_id
df1=df1.replace({'date': date})
df1['ticker']= earning_call_id
df1=df1.replace({'ticker': id_mapping})
df1= df1.sort_values(by=['id'], ascending=False)
display(df1[:1000])

,id,date,ticker
751,4512747,2022-05-18,AYX
76,4512744,2022-05-18,TEF
1328,4512738,2022-05-18,DTRUY
7,4512728,2022-05-17,ENGIY
791,4512725,2022-05-17,DOCS
...,...,...,...
513,4510142,2022-05-11,EVAX
1168,4510141,2022-05-11,PANL
1115,4510140,2022-05-11,AGFY
371,4510128,2022-05-11,XERS


In [24]:
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_Symbol = table[0]
df_Symbol.to_csv('S&P500-Info.csv')
df_Symbol.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
Symbol=df["Symbol"].values
print(Symbol)

['MMM' 'AOS' 'ABT' 'ABBV' 'ABMD' 'ACN' 'ATVI' 'ADM' 'ADBE' 'ADP' 'AAP'
 'AES' 'AFL' 'A' 'AIG' 'APD' 'AKAM' 'ALK' 'ALB' 'ARE' 'ALGN' 'ALLE' 'LNT'
 'ALL' 'GOOGL' 'GOOG' 'MO' 'AMZN' 'AMCR' 'AMD' 'AEE' 'AAL' 'AEP' 'AXP'
 'AMT' 'AWK' 'AMP' 'ABC' 'AME' 'AMGN' 'APH' 'ADI' 'ANSS' 'ANTM' 'AON'
 'APA' 'AAPL' 'AMAT' 'APTV' 'ANET' 'AIZ' 'T' 'ATO' 'ADSK' 'AZO' 'AVB'
 'AVY' 'BKR' 'BALL' 'BAC' 'BBWI' 'BAX' 'BDX' 'WRB' 'BRK.B' 'BBY' 'BIO'
 'TECH' 'BIIB' 'BLK' 'BK' 'BA' 'BKNG' 'BWA' 'BXP' 'BSX' 'BMY' 'AVGO' 'BR'
 'BRO' 'BF.B' 'CHRW' 'CDNS' 'CZR' 'CPT' 'CPB' 'COF' 'CAH' 'KMX' 'CCL'
 'CARR' 'CTLT' 'CAT' 'CBOE' 'CBRE' 'CDW' 'CE' 'CNC' 'CNP' 'CDAY' 'CERN'
 'CF' 'CRL' 'SCHW' 'CHTR' 'CVX' 'CMG' 'CB' 'CHD' 'CI' 'CINF' 'CTAS' 'CSCO'
 'C' 'CFG' 'CTXS' 'CLX' 'CME' 'CMS' 'KO' 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG'
 'COP' 'ED' 'STZ' 'CEG' 'COO' 'CPRT' 'GLW' 'CTVA' 'COST' 'CTRA' 'CCI'
 'CSX' 'CMI' 'CVS' 'DHI' 'DHR' 'DRI' 'DVA' 'DE' 'DAL' 'XRAY' 'DVN' 'DXCM'
 'FANG' 'DLR' 'DFS' 'DISH' 'DIS' 'DG' 'DLTR' 'D' 'DPZ' 'DOV' 'DOW

In [16]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

tickers = df1['ticker'].tolist()
start = datetime.datetime(2019,1,1)
end = datetime.datetime(2021,12,31)

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in tickers[:500]:  
    
    # print the symbol which is being downloaded
    print( str(tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, interval = "1m", progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None


0 : AYX,
1 Failed download:
- AYX: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
1 : TEF,
1 Failed download:
- TEF: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
2 : DTRUY,
1 Failed download:
- DTRUY: 1m data not available for startTime=1639405800 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
3 : ENGIY,
1 Failed download:
- ENGIY: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
4 : DOCS,
1 Failed download:
- DOCS: 1m data not available for startTime=1624541400 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
5 : LVLU,
1 Failed download:
- LVLU: 1m data not available f

44 : LGVN,
1 Failed download:
- LGVN: 1m data not available for startTime=1613140200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
45 : SBET,
1 Failed download:
- SBET: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
46 : NGS,
1 Failed download:
- NGS: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
47 : FTK,
1 Failed download:
- FTK: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
48 : TEDU,
1 Failed download:
- TEDU: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
49 : MITQ,
1 Failed download:
- MITQ: 1m data not available

88 : VOD,
1 Failed download:
- VOD: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
89 : GVP,
1 Failed download:
- GVP: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
90 : YALA,
1 Failed download:
- YALA: 1m data not available for startTime=1601472600 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
91 : DNTUF,
1 Failed download:
- DNTUF: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
92 : FXLV,
1 Failed download:
- FXLV: 1m data not available for startTime=1626355800 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
93 : MVST,
1 Failed download:
- MVST: 1m data not availab

132 : PEYE,
1 Failed download:
- PEYE: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
133 : KULR,
1 Failed download:
- KULR: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
134 : AXDX,
1 Failed download:
- AXDX: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
135 : ADMP,
1 Failed download:
- ADMP: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
136 : SSYS,
1 Failed download:
- SSYS: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
137 : CWBR,
1 Failed download:
- CWBR: 1m data not

176 : WYY,
1 Failed download:
- WYY: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
177 : VEV,
1 Failed download:
- VEV: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
178 : KBNT,
1 Failed download:
- KBNT: 1m data not available for startTime=1597239000 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
179 : REKR,
1 Failed download:
- REKR: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
180 : MARK,
1 Failed download:
- MARK: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
181 : XSPA,
1 Failed download:
- XSPA: 1m data not ava

...

308 : LVOX,
1 Failed download:
- LVOX: 1m data not available for startTime=1555594200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
309 : BRP,
1 Failed download:
- BRP: 1m data not available for startTime=1571923800 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
310 : APEI,
1 Failed download:
- APEI: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
311 : TACT,
1 Failed download:
- TACT: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
312 : RLGT,
1 Failed download:
- RLGT: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
313 : ADV,
1 Failed download:
- ADV: 1m data not ava

352 : DXPE,
1 Failed download:
- DXPE: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
353 : TPG,
1 Failed download:
- TPG: Data doesn't exist for startDate = 1546297200, endDate = 1640905200
354 : CMAX,
1 Failed download:
- CMAX: 1m data not available for startTime=1600695000 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
355 : GCMG,
1 Failed download:
- GCMG: 1m data not available for startTime=1549463400 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
356 : HSON,
1 Failed download:
- HSON: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
357 : CTG,
1 Failed download:
- CTG: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m gra

397 : CCO,
1 Failed download:
- CCO: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
398 : MLNK,
1 Failed download:
- MLNK: 1m data not available for startTime=1627479000 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
399 : ISSC,
1 Failed download:
- ISSC: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
400 : STER,
1 Failed download:
- STER: 1m data not available for startTime=1632403800 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
401 : PFSW,
1 Failed download:
- PFSW: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
402 : STVN,
1 Failed download:
- STVN: 1m data not a

441 : NPCE,
1 Failed download:
- NPCE: 1m data not available for startTime=1619098200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
442 : PKE,
1 Failed download:
- PKE: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
443 : LIDR,
1 Failed download:
- LIDR: 1m data not available for startTime=1610375400 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
444 : IFSPF,
1 Failed download:
- IFSPF: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
445 : SMSOF,
1 Failed download:
- SMSOF: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
446 : BKTI,
1 Failed download:
- BKTI: 1m data n

486 : KINS,
1 Failed download:
- KINS: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
487 : PRPH,
1 Failed download:
- PRPH: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
488 : HUMA,
1 Failed download:
- HUMA: 1m data not available for startTime=1606833000 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
489 : DDI,
1 Failed download:
- DDI: 1m data not available for startTime=1630416600 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
490 : RGF,
1 Failed download:
- RGF: 1m data not available for startTime=1636119000 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
491 : EMBK,
1 Failed download:
- EMBK: 1m data not ava

In [14]:
stock_final

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2022-05-16,58.520000,59.865002,55.709999,55.799999,55.799999,703300,AYX
2022-05-17,57.000000,58.240002,54.119999,56.200001,56.200001,587900,AYX
2022-05-18,55.400002,56.230000,52.790001,53.570000,53.570000,678400,AYX
2022-05-16,4.890000,4.950000,4.860000,4.950000,4.950000,2578500,TEF
2022-05-17,4.980000,5.050000,4.960000,5.050000,5.050000,1598100,TEF
...,...,...,...,...,...,...,...
2022-05-17,13.960000,14.300000,13.170000,13.290000,13.290000,119300,AOMR
2022-05-18,13.290000,13.650000,13.220000,13.300000,13.300000,96700,AOMR
2022-05-16,3.500000,3.680000,3.490000,3.620000,3.620000,123800,AAIC


In [10]:
# Preprocessing
n= 10
texts = [processing(x) for x in texts][:n]

In [11]:
#N=len(texts)

for i in range(n):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

In [12]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)

In [16]:
number_words = np.zeros(n)
for i in range(n):
    number_words[i] = len(texts[i])
df['number_words']=number_words
df['id'] = earning_call_id[:n]
df['date']=earning_call_id[:n]
df=df.replace({'date': date})
df['ticker']= earning_call_id[:n]
df=df.replace({'ticker': id_mapping})
df= df.sort_values(by=['id'], ascending=False)
display(df)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,number_words,id,date,ticker
7,0.008483853311439518,0.0481663929939792,0.0021893814997263274,0.010673234811165846,0.02435686918445539,0.029556650246305417,0.09688013136288999,0.014230979748221127,0.00875752599890531,0.05172413793103448,28106.0,4512728,2022-05-17,ENGIY
2,0.008119079837618403,0.05087956698240866,0.0040595399188092015,0.011096075778078484,0.020838971583220567,0.022192151556156968,0.09769959404600811,0.009201623815967524,0.009742895805142085,0.04817320703653586,27264.0,4512525,2022-05-17,SONVF
5,0.011382113821138212,0.04390243902439024,0.0040650406504065045,0.00975609756097561,0.01788617886178862,0.023577235772357725,0.10650406504065041,0.01056910569105691,0.012195121951219513,0.05853658536585366,9243.0,4510888,2022-05-12,HCDI
0,0.0046248715313463515,0.046762589928057555,0.0005138746145940391,0.0046248715313463515,0.018499486125385406,0.013874614594039054,0.0750256937307297,0.0051387461459403904,0.013874614594039054,0.03597122302158273,14598.0,4510424,2022-05-11,LNNNF
3,0.004914933837429111,0.052930056710775046,0.0011342155009451795,0.007183364839319471,0.022684310018903593,0.019281663516068054,0.09640831758034027,0.011720226843100189,0.010964083175803403,0.05330812854442344,19476.0,4510272,2022-05-11,VIV
8,0.011375947995666305,0.048212351029252434,0.005417118093174431,0.009209100758396533,0.024918743228602384,0.029794149512459372,0.1056338028169014,0.013542795232936078,0.010292524377031419,0.06717226435536294,14245.0,4509536,4509536,4509536
1,0.01146384479717813,0.0582010582010582,0.007495590828924162,0.007054673721340388,0.040123456790123455,0.029541446208112873,0.1027336860670194,0.009259259259259259,0.021164021164021163,0.06128747795414462,16564.0,4509509,4509509,4509509
9,0.009034589571502324,0.05085183273102736,0.002065049044914817,0.006969540526587507,0.02194114610221993,0.01961796592669076,0.08466701084150749,0.005678884873515746,0.014197212183789365,0.059886422302529684,28539.0,4509394,4509394,4509394
6,0.009439200444197668,0.06274292059966685,0.000555247084952804,0.011660188784008884,0.03387007218212104,0.02387562465297057,0.1254858411993337,0.012770682953914493,0.014436424208772903,0.08217656857301499,13526.0,4509360,4509360,4509360
4,0.0055514433752775726,0.05773501110288675,0.0025906735751295338,0.011102886750555145,0.021095484826054774,0.015173945225758698,0.10510732790525537,0.007031828275351592,0.014433752775721688,0.058105107327905255,20509.0,4509199,4509199,4509199


In [14]:
print(date.get('4509199'))

None


# A faire pour cette méthode : 
- classer les earning calls par quarter et par stock index
- importer les stock returns et garder seulement ceux des earning calls (mapping permno/ stock index)

# liens utiles

https://betterprogramming.pub/unlocking-emotions-in-text-using-python-6d062b48d71f